In [1]:
import cirq
import openfermion as of
import openfermionpyscf as ofpyscf

/opt/anaconda3/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [138]:
import os

from openfermion.chem import make_atomic_ring, MolecularData

from openfermionpsi4 import run_psi4

In [139]:
ofpyscf.generate_molecular_hamiltonian?

Signature:
ofpyscf.generate_molecular_hamiltonian(
    geometry,
    basis,
    multiplicity,
    charge=0,
    n_active_electrons=None,
    n_active_orbitals=None,
)
Docstring:
Generate a molecular Hamiltonian with the given properties.

Args:
    geometry: A list of tuples giving the coordinates of each atom.
        An example is [('H', (0, 0, 0)), ('H', (0, 0, 0.7414))].
        Distances in angstrom. Use atomic symbols to
        specify atoms.
    basis: A string giving the basis set. An example is 'cc-pvtz'.
        Only optional if loading from file.
    multiplicity: An integer giving the spin multiplicity.
    charge: An integer giving the charge.
    n_active_electrons: An optional integer specifying the number of
        electrons desired in the active space.
    n_active_orbitals: An optional integer specifying the number of
        spatial orbitals desired in the active space.

Returns:
    The Hamiltonian as an InteractionOperator.
File:      /opt/anaconda3/lib/python3.9

In [156]:
molecule = molcls[0]
geometry = molecule['geometry']
charge = 0
multplicity = 1
ham_ferm_op = ofpyscf.generate_molecular_hamiltonian(geometry = geometry, basis = 'sto-3g', multiplicity=1,
                                       charge = 0, n_active_orbitals=5)

ham_ferm_op

() 0.995380044366418
((0, 1), (0, 0)) -4.728441979871503
((0, 1), (2, 0)) 0.10568647604356646
((0, 1), (4, 0)) 0.16702128382098483
((1, 1), (1, 0)) -4.728441979871503
((1, 1), (3, 0)) 0.10568647604356646
((1, 1), (5, 0)) 0.16702128382098483
((2, 1), (0, 0)) 0.1056864760435666
((2, 1), (2, 0)) -1.4946161051733082
((2, 1), (4, 0)) 0.03303590268975044
((3, 1), (1, 0)) 0.1056864760435666
((3, 1), (3, 0)) -1.4946161051733082
((3, 1), (5, 0)) 0.03303590268975044
((4, 1), (0, 0)) 0.16702128382098497
((4, 1), (2, 0)) 0.03303590268975035
((4, 1), (4, 0)) -1.1258901743584766
((5, 1), (1, 0)) 0.16702128382098497
((5, 1), (3, 0)) 0.03303590268975035
((5, 1), (5, 0)) -1.1258901743584766
((6, 1), (6, 0)) -1.1362769993713195
((7, 1), (7, 0)) -1.1362769993713195
((8, 1), (8, 0)) -1.1362769993713202
((9, 1), (9, 0)) -1.1362769993713202
((0, 1), (0, 1), (0, 0), (0, 0)) 0.8292756027716603
((0, 1), (0, 1), (0, 0), (2, 0)) -0.05597289155814363
((0, 1), (0, 1), (0, 0), (4, 0)) -0.06926553349587015
((0, 1), 

In [3]:
# I will also try using pennylane
import pennylane as qml
import numpy as np

In [4]:
with open("structures.txt", "r") as f:
    lines = f.readlines()

In [5]:
def get_molecules(lines):
    
    molecules = []
    
    symb = ""
    for l in range(len(lines)):
        
        line = lines[l].strip()
        
        if line == "start":
            
            molecule = {"symb": lines[l+1].strip()}
            molecule["geometry"] = []
        
        elif line == "end":
            
            molecules.append(molecule)
        
        
        elif line.split(":")[0] == "charge":
            
            molecule["charge"] = float(line.split(":")[1])
        
        elif line.split(":")[0] == "multiplicity":
            
            molecule["multiplicity"] = float(line.split(":")[1])
        
        else:
            
            if lines[l-1] != "start\n":
            
                line_splitted = line.split("\t")

                atom = line_splitted[0][:-1]
                position = [float(u) for u in line_splitted[1:]]

                molecule["geometry"].append([atom, position])

        
    return molecules
molcls = get_molecules(lines)

In [161]:
def get_ham(molecule):
    
    basis = 'sto-3g'
    geometry = molecule["geometry"]
    multiplicity = molecule["multiplicity"]
    charge = molecule["charge"]
    
    # print("------------")
    # print("basis = ", basis)
    # print("geometry = ", geometry)
    # print("multiplicity = ", multiplicity)
    # print("charge = ", charge)
    # print("--------------")
    
    # compute hamiltonian
    #multiplicity = 3
    hamiltonian = ofpyscf.generate_molecular_hamiltonian(
    geometry, basis, multiplicity, charge)

    # Convert to a FermionOperator
    hamiltonian_ferm_op = of.get_fermion_operator(hamiltonian)

    hamiltonian_ferm_op.compress()
    
    # ham_ferm_to_qubits = of.transforms.symmetry_conserving_bravyi_kitaev(hamiltonian_ferm_op, active_orbitals=6,active_fermions=2)
    
    # return ham_ferm_to_qubits
    
    try:
        # use bravyi-kitaev transf.
        hamiltonian_bk = of.transforms.symmetry_conserving_bravyi_kitaev(hamiltonian_ferm_op, active_orbitals=5,\
                                                         active_fermions=3)
    except KeyError:
        # do simple bravyi kitaev
        # print("The symmetry conserving BK failed with error KeyError")
        # print("I proceed with Simple BK")
        hamiltonian_bk = of.transforms.bravyi_kitaev(hamiltonian_ferm_op)
    
    return hamiltonian_bk

In [162]:
molecule = molcls[0]
print("The molecule: {0}".format(molecule["symb"]))
hamiltonian = get_ham(molecule)
find_nqbits(hamiltonian)

The molecule: LiH


12

In [108]:
def parse_key(key, n):
    
    if key == ():
        
        return "I"*n
    
    q = 0
    pauli_string = ""
    for item in key:
        
        while q < item[0]:
            
            pauli_string += "I"
            q += 1
        
        pauli_string += item[1]
        q += 1
    
    
    pauli_string += "I"*(n - q)
        
    return pauli_string

In [109]:
def find_nqbits(hamiltonian):
    
    n = 0
    
    for key in hamiltonian.terms.keys():
        
        for tmp in key:
            
            ntmp = tmp[0]
            
            if ntmp > n:
                
                n = ntmp
    
    return n + 1

In [110]:
molecule = molcls[0]
print("The molecule: {0}".format(molecule["symb"]))
hamiltonian = get_ham(molecule)
find_nqbits(hamiltonian)

The molecule: LiH


12

In [10]:
print(f"The number of terms: {len(hamiltonian.terms)}")

The number of terms: 631


In [11]:
def ham_to_pauli_strings(hamiltonian, filename):
    
    n = find_nqbits(hamiltonian)
    
    with open(filename, "w") as f:
    
        for key, item in hamiltonian.terms.items():
            
            pstring = parse_key(key, n)
            
            f.write("{0},{1}\n".format(pstring, item.real))
    
    return

In [12]:
def find_trivial_qbits(hamiltonian):
    
    n = find_nqbits(hamiltonian)
    
    qbits = []
    
    for key, item in hamiltonian.terms.items():
        
        pstring = parse_key(key, n)
        
        for q in range(n):
            
            if pstring[q] != "I" and not q in qbits:
                
                print(pstring)
                
                qbits.append(q)
    return qbits

In [157]:
def get_ham_qml(molecule):
    
    symbols = [atom[0] for atom in molecule["geometry"]]
    coordinates = []
    for atom in molecule["geometry"]:
        coordinates += atom[1] 
    coordinates = np.array(coordinates)
    
    
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates,\
                                                active_orbitals = 5,\
                                                mult = 1,\
                                                 mapping="bravyi_kitaev",\
                                                    basis = "sto-3g")
    
    return molecule["symb"], H, qubits


In [158]:
symb, hh, qubits = get_ham_qml(molcls[0])
symb, qubits, hh

('LiH', 10, <Hamiltonian: terms=276, wires=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]>)

### Use Qiskit